**Imports**

In [8]:
 # PIL: Python image library
from PIL import Image

**Global Variables**

In [ ]:
threshold = 200

**Function Definitions**

In [16]:
# determine_black_or_white: Takes in a pixel value (0-255) and uses threshold to determine whether pixel should be changed to white or black
def determine_black_or_white(pixel_val):
    # If the pixel is close enough to white, assign white
    if pixel_val > threshold:
        return 255
    # Otherwise assign black
    else:
        return 0

**Main Code**

In [13]:
# Load Image as Object
img = Image.open('./original_photos/Plocamium.pacificum_LC_P_SA_26Jan2024_2.JPG')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3648x2736 at 0x23436584C10>


In [17]:
# Convert Image to Black-White
r = img.convert('L').point(determine_black_or_white(), mode='1')

TypeError: determine_black_or_white() missing 1 required positional argument: 'pixel_val'